In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


from QUANTAXIS.QAUtil import DATABASE
from QUANTAXIS.QAUtil import  trade_date_sse
from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date,
    QA_util_get_next_trade_date,
    QA_util_if_tradetime
)

import inspect
import ind.Alpha101 as toys

from sklearn import linear_model

# %load_ext autoreload
# %autoreload 2
# %aimport tools.Cacher

# def assemble_stocks_by_codes(codes):
#     files = list(map(lambda x:x+'_train_qfq',l))
#     return pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),files))).sort_index(level=0)
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates



import ind.Alpha191 as a191
from scipy import stats
import statsmodels.api as sm

# import cpuinfo
# if 'ntel' in cpuinfo.get_cpu_info()['brand_raw']:
# from sklearnex import patch_sklearn, unpatch_sklearn
# unpatch_sklearn() ##注意，少量数据的线性回归没有优势。慎用，存在内存泄露

from sklearn import linear_model


%load_ext autoreload
%autoreload 2
%aimport ind.Alpha191

def pretreate_data(data):
    returns = smpl.get_current_return(data,'close')
    returns.name = 'returns'
    ret_forward = smpl.get_forward_return(data,'close')
    ret_forward.name = 'ret_forward'
    # {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
    data = pd.concat([data, returns, ret_forward], axis=1)
    data = data.assign(vwap=data.amount/(data.volume*100))
    data.rename(columns = {"open":"Open",'market_value':'cap','industry':'ind'}, inplace=True)
    data = smpl.add_report_inds(data,'netAssetsPerShare')
    data['pb']=data['close']/data['netAssetsPerShare']
    
    df_ids = data.index.get_level_values(0)
    
    benchmark = smpl.get_benchmark('沪深300', start=df_ids.min(), end=df_ids.max()).data
    benchmark = benchmark[['open','close']].reset_index('code',drop=True)
    data['bm_index_open'] = df_ids.map(benchmark['open'])
    data['bm_index_close'] = df_ids.map(benchmark['close'])
    return data
    # return pd.concat([data, close_ind, vwap_ind, low_ind, high_ind, volume_ind, adv20_ind, adv40_ind, adv81_ind, co_mixed_ind, oh_mixed_ind, lv_mixed_ind], axis=1)

def assemble_stocks_by_codes(codes):
    files = list(map(lambda x:x+'_train_qfq',codes))
    return pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),files))).sort_index(level=0)



from tools import QAAdapter as qaadapter
from factorBase import FactorBase

class alpha191(FactorBase):

    def calc(self, data) -> pd.DataFrame:
        import inspect
        import ind.Alpha191 as a191
        from base.JuUnits import excute_for_multidates
        """

        the example is just a day datasource, u can use the min data to generate a day-frequence factor

        the factor should be in day frequence 
        """
        i = int(self.factor_name.split('_')[1])
        fun_name = 'alpha'+ '0'*(3- int(np.log10(i)+1)) +str(i)
        print("fun_name ", fun_name)

        fun_name = 'alpha'+ '0'*(3- int(np.log10(i)+1)) +str(i)
        if fun_name != 'alpha030':
            factor = excute_for_multidates(data, lambda x: getattr(a191, fun_name)(x) ,level=1)
        else:
            factor = getattr(a191, fun_name)(data)
            
        factor.columns = ['factor']

        factor = pd.DataFrame(factor)
        factor = factor.replace([-np.inf, np.inf], 0).fillna(value=0)
        factor.columns = ['factor']

        return factor.reset_index().dropna()


jqdatasdk not installed
you are using non-interactive mdoel quantaxis


In [ ]:
# ids = [11,24,38,41,42,47,57,69,80,82,83,88,93,97]
# ids = [11,24]

# l = smpl.get_codes_from_blockname('沪深300', sse='all')
# df_all =  assemble_stocks_by_codes(l) #文件已合并，不再适用
# df_treated = pretreate_data(df_all)
from tools import QAAdapter as qaadapter
import QUANTAXIS as QA
import tools.Sample_Tools as smpl

from base.Constants import trade_date_sse
gap = 90
end_date = '2023-07-21'
start_date = trade_date_sse[trade_date_sse.index(end_date)-gap]
# stock_list  = qaadapter.QA_adapter_get_code_from_block('sw_l1', '国防军工')
stock_list = smpl.get_codes_by_market(sse='all',only_main=True,filter_st=True)

data = QA.QA_fetch_stock_day_adv(stock_list, start_date, end_date).data
data = smpl.add_marketvalue_industry(data)

# #读取已经缓存的复权数据，并预处理
smpl.optimize_data_type(data)
df_treated = pretreate_data(data)

ids = np.arange(1,191)
np.random.shuffle(ids)

def generate_alpha191_factors(fun_ids, df=None):

    from qaenv import (clickhouse_ip, clickhouse_password, clickhouse_port,
                      clickhouse_user)
    for i in fun_ids:
        
        factor_name = 'alpha191_' + str(i)
        factor_base = alpha191(factor_name, host=clickhouse_ip, port=clickhouse_port, user=clickhouse_user, password=clickhouse_password)
        factor_base.update_to_database(df)


worker=6
task = task_chunk_split(ids, worker)
results = parallal_task(worker, generate_alpha191_factors, task, df=df_treated)


In [3]:
from qaenv import (clickhouse_ip, clickhouse_password, clickhouse_port,
                  clickhouse_user)
    
from tools import QAAdapter as qaadapter
import QUANTAXIS as QA
import tools.Sample_Tools as smpl

from base.Constants import trade_date_sse
gap = 90
end_date = '2023-07-21'
start_date = trade_date_sse[trade_date_sse.index(end_date)-gap]
# stock_list  = qaadapter.QA_adapter_get_code_from_block('sw_l1', '国防军工')
stock_list = smpl.get_codes_by_market(sse='all',only_main=True,filter_st=True)

data = QA.QA_fetch_stock_day_adv(stock_list, start_date, end_date).data
data = smpl.add_marketvalue_industry(data)

# #读取已经缓存的复权数据，并预处理
smpl.optimize_data_type(data)
df = pretreate_data(data)

ids = np.arange(1,191)
np.random.shuffle(ids)

i = 110
fun_name = 'alpha'+ '0'*(3- int(np.log10(i)+1)) +str(i)
# print(df)
        
factor_name = 'alpha191_' + str(i)
factor_base = alpha191(factor_name, host=clickhouse_ip, port=clickhouse_port, user=clickhouse_user, password=clickhouse_password)
factor_base.update_to_database(df)




fun_name  alpha110
